In [18]:
import param
import panel as pn

pn.extension('katex')

In [19]:
pipeline = pn.pipeline.Pipeline()

In [20]:
class Stage1(param.Parameterized):

    a = param.Number(default=5, bounds=(0, 10))

    b = param.Number(default=5, bounds=(0, 10))
    
    ready = param.Boolean(default=False, precedence=-1)

    @param.output(('c', param.Number), ('d', param.Number))
    def output(self):
        return self.a * self.b, self.a ** self.b
    
    @param.depends('a', 'b')
    def view(self):
        c, d = self.output()
        return pn.pane.LaTeX('${a} * {b} = {c}$\n${a}^{{{b}}} = {d}$'.format(
            a=self.a, b=self.b, c=c, d=d), style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)

stage1 = Stage1()
stage1.panel()

In [21]:
stage1.param.outputs()

In [22]:
pipeline.add_stage('Stage 1', stage1)

In [23]:
class Stage2(param.Parameterized):
    
    c = param.Number(default=5, bounds=(0, None))

    exp = param.Number(default=0.1, bounds=(0, 3))
    
    @param.depends('c', 'exp')
    def view(self):
        return pn.pane.LaTeX('${%s}^{%s}={%.3f}$' % (self.c, self.exp, self.c**self.exp),
                             style={'font-size': '2em'})

    def panel(self):
        return pn.Row(self.param, self.view)
    
stage2 = Stage2(c=stage1.output()[0])
stage2.panel()

In [7]:
pipeline.add_stage('Stage 2', stage2)

In [8]:
print(pipeline)

In [9]:
pipeline = pn.pipeline.Pipeline(debug=True)
pipeline.add_stage('Stage 1', Stage1())
pipeline.add_stage('Stage 2', Stage2)

pipeline

In [10]:
pipeline = pn.pipeline.Pipeline([('Stage 1', Stage1), ('Stage 2', Stage2)])
pipeline

In [11]:
class Input(param.Parameterized):
    
    value1 = param.Integer(default=2)

    value2 = param.Integer(default=3)
    
    def panel(self):
        return pn.Row(self.param.value1, self.param.value2)

class Multiply(Input):
    
    def panel(self):
        return '%.3f * %.3f' % (self.value1, self.value2)

    @param.output('result')
    def output(self):
        return self.value1 * self.value2
    
class Add(Input):
    
    def panel(self):
        return '%d + %d' % (self.value1, self.value2)
    
    @param.output('result')
    def output(self):
        return self.value1 + self.value2
    
class Result(Input):
    
    result = param.Number(default=0)

    def panel(self):
        return self.result
    
dag = pn.pipeline.Pipeline()

dag.add_stage('Input', Input)
dag.add_stage('Multiply', Multiply)
dag.add_stage('Add', Add)
dag.add_stage('Result', Result)

In [12]:
dag.define_graph({'Input': ('Multiply', 'Add'), 'Multiply': 'Result', 'Add': 'Result'})

In [13]:
dag

In [14]:
pn.Column(
    pn.Row(dag.title, pn.layout.HSpacer(), dag.buttons),
    dag.network,
    dag.stage
)

In [15]:
class AutoInput(Input):

    operator = param.Selector(default='Add', objects=['Multiply', 'Add'])

    ready = param.Boolean(default=False)
    
    def panel(self):
        button = pn.widgets.Button(name='Go', button_type='success')
        button.on_click(lambda event: setattr(self, 'ready', True))
        widgets = pn.Row(self.param.value1, self.param.operator, self.param.value2)
        for w in widgets:
            w.width = 85
        return pn.Column(widgets, button)

class AutoMultiply(Multiply):
    
    ready = param.Boolean(default=True)

class AutoAdd(Add):
    
    ready = param.Boolean(default=True)

In [16]:
dag = pn.pipeline.Pipeline() # could instead set ready_parameter='ready' and auto_advance=True globally here

dag.add_stage('Input',    AutoInput,    ready_parameter='ready', auto_advance=True, next_parameter='operator')
dag.add_stage('Multiply', AutoMultiply, ready_parameter='ready', auto_advance=True)
dag.add_stage('Add',      AutoAdd,      ready_parameter='ready', auto_advance=True)
dag.add_stage('Result',   Result)

dag.define_graph({'Input': ('Multiply', 'Add'), 'Multiply': 'Result', 'Add': 'Result'})

In [17]:
pn.Column(
    dag.title,
    dag.network,
    dag.stage
)